##This is the cool project Airline Schedule

###Construct Graph for routes, Node should have 6, airport with x gates has x Node, each node has ###'Ground_Time' attribute, make it 2 minutes more than min. DAL1 and DAL2 has no edge, same as HOU1,HOU2 and HOU3


1. 360(600 morning)<=DTM<ATM<=1320(2200 night)

2. AUS->DAL  50 Edge_LEN
   DAL->AUS  50
   AUS->HOU  45
   HOU->AUS  45
   DAL->HOU  65
   HOU->DAL  65

3. AUS1  GTM_END - GTM_START >= 25 or GTM_END, GTM_START are 0 
   DAL1  GTM_END - GTM_START >= 30
   DAL2  GTM_END - GTM_START >= 30
   HOU1  GTM_END - GTM_START >= 35
   HOU2  GTM_END - GTM_START >= 35
   HOU3  GTM_END - GTM_START >= 35

4. DTM < GTM_START
   ATM > GTM_END

5. DTM = ATM + (GTM_END-GTM_START)

6. ATM =  DTM + Edge_LEN

7. AUS T1
   DAL T5
   DAL T6
   HOU T2
   HOU T3
   HOU T4

In [19]:
import networkx as nx

routes = nx.Graph()

#ground_time as attribute
routes.add_node("AUS1", ground_time = 27)
routes.add_node("DAL1", ground_time = 32)
routes.add_node("DAL2", ground_time = 32)
routes.add_node("HOU1", ground_time = 37)
routes.add_node("HOU2", ground_time = 37)
routes.add_node("HOU3", ground_time = 37)

#print(routes.nodes(data=True))
#print(routes.node["DAL1"]["ground_time"])

#fly_time as weight for edge
routes.add_edge("AUS1","DAL1", fly_time = 50)
routes.add_edge("DAL1","AUS1", fly_time = 50)
routes.add_edge("AUS1","DAL2", fly_time = 50)
routes.add_edge("DAL2","AUS1", fly_time = 50)

routes.add_edge("AUS1","HOU1", fly_time = 45)
routes.add_edge("HOU1","AUS1", fly_time = 45)
routes.add_edge("AUS1","HOU2", fly_time = 45)
routes.add_edge("HOU2","AUS1", fly_time = 45)
routes.add_edge("AUS1","HOU3", fly_time = 45)
routes.add_edge("HOU3","AUS1", fly_time = 45)

routes.add_edge("HOU1","DAL1", fly_time = 65)
routes.add_edge("DAL1","HOU1", fly_time = 65)
routes.add_edge("HOU2","DAL1", fly_time = 65)
routes.add_edge("DAL1","HOU2", fly_time = 65)
routes.add_edge("HOU3","DAL1", fly_time = 65)
routes.add_edge("DAL1","HOU3", fly_time = 65)

routes.add_edge("HOU1","DAL2", fly_time = 65)
routes.add_edge("DAL2","HOU1", fly_time = 65)
routes.add_edge("HOU2","DAL2", fly_time = 65)
routes.add_edge("DAL2","HOU2", fly_time = 65)
routes.add_edge("HOU3","DAL2", fly_time = 65)
routes.add_edge("DAL2","HOU3", fly_time = 65)

#print(routes.edges()[:5])
#print(routes.neighbors("HOU3"))
#print(routes.degree())

nx.clustering(routes)


{'AUS1': 0.6,
 'DAL1': 0.5,
 'DAL2': 0.5,
 'HOU1': 0.6666666666666666,
 'HOU2': 0.6666666666666666,
 'HOU3': 0.6666666666666666}

In [15]:
from pandas import DataFrame

Optimal_Schedule = [
    {'tail_num':'T1', 'org':'DAL', 'dest':'AUS', 'd_time':360, 'a_time':410},
    {'tail_num':'T1', 'org':'AUS', 'dest':'HOU', 'd_time':600, 'a_time':645}
]

#define minutes to military time
def convert_military_time (var): 
    return var//60 * 100 + var % 60

#define
def get_var(var):
    variable = []
    
    if var == 'd_time':
        for i in Optimal_Schedule:
            value = convert_military_time(i[var])      
            variable.append(value)
    elif var == 'a_time':
        for i in Optimal_Schedule:
            value = convert_military_time(i[var])      
            variable.append(value)
    else:
        for i in Optimal_Schedule:
            value = i[var] 
            variable.append(value)
           
    return variable

#use function
tail_num = get_var('tail_num')
org = get_var('org')
dest = get_var ('dest')
d_time = get_var ('d_time')
a_time = get_var ('a_time')

data = {'tail_number':tail_num, 'origin':org, 'destination':dest, 'departure_time':d_time, 'arrival_time':a_time}
frame = DataFrame(data)
frame.to_csv('flight_schedule.csv', mode = 'w', index=False, columns=['tail_number', 'origin', 'destination', 'departure_time','arrival_time'])